# Bronze Layer: Cargo Manifest with COPY INTO
**Ingestion Pattern**: COPY INTO

**Features**:
 - Idempotent loads (same file won't be loaded twice)
 - Efficient for batch loads
 - Automatic schema inference
 - File-level tracking

In [0]:
from pyspark.sql.functions import *
from datetime import date
import logging

# Import utilities
import sys
sys.path.append("../utils")
from logging_utils import get_logger
from validation_utils import validate_manifest_data

logger = get_logger("bronze_manifest_copyinto")

In [0]:
# Get parameters
dbutils.widgets.text("catalog_name", "cargo_fleet_dev", "Catalog")
dbutils.widgets.text("load_date", str(date.today()), "Load Date")

catalog_name = dbutils.widgets.get("catalog_name")
load_date = dbutils.widgets.get("load_date")

bronze_table = f"{catalog_name}.bronze.cargo_manifest_raw"
source_path = f"/Volumes/{catalog_name}/bronze/manifest_landing"

logger.info(f"Starting COPY INTO for load date: {load_date}")

In [0]:
try:
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {bronze_table} (
            manifest_id STRING,
            ship_id STRING,
            voyage_number STRING,
            load_port STRING,
            destination_port STRING,
            departure_date DATE,
            estimated_arrival_date DATE,
            container_id STRING,
            cargo_type STRING,
            cargo_weight_kg DOUBLE,
            cargo_value_usd DOUBLE,
            shipper_name STRING,
            consignee_name STRING,
            special_handling STRING,
            load_timestamp TIMESTAMP,
            ingestion_timestamp TIMESTAMP,
            source_file STRING
        )
        USING DELTA
        PARTITIONED BY (departure_date)
        COMMENT 'Raw cargo manifest data from daily batch loads'
    """)
    
    logger.info(f"✓ Target table created/verified: {bronze_table}")
    
except Exception as e:
    logger.error(f"Failed to create target table: {str(e)}")
    raise

In [0]:
file_pattern = f"{source_path}/manifest_{load_date.replace('-', '')}.parquet"

try:
    # Execute COPY INTO command
    result = spark.sql(f"""
        COPY INTO {bronze_table}
        FROM (
            SELECT 
                *,
                current_timestamp() as ingestion_timestamp,
                _metadata.file_path as source_file
            FROM '{file_pattern}'
        )
        FILEFORMAT = PARQUET
        FORMAT_OPTIONS ('mergeSchema' = 'true')
        COPY_OPTIONS ('mergeSchema' = 'true')
    """)
    
    # Get load statistics
    stats = result.collect()[0]
    num_rows_loaded = stats['num_affected_rows']
    
    logger.info(f"✓ COPY INTO completed successfully")
    logger.info(f"  - Rows loaded: {num_rows_loaded}")
    logger.info(f"  - Target table: {bronze_table}")
    
    # Display result
    display(result)
    
except Exception as e:
    logger.error(f"COPY INTO failed: {str(e)}")
    
    # Check if file already loaded (idempotent behavior)
    if "already loaded" in str(e).lower():
        logger.warning(f"File already loaded (idempotent): {file_pattern}")
    else:
        raise

In [0]:
# Count records loaded today
df_today = spark.sql(f"""
    SELECT COUNT(*) as record_count,
           COUNT(DISTINCT manifest_id) as manifest_count,
           COUNT(DISTINCT ship_id) as ship_count
    FROM {bronze_table}
    WHERE DATE(ingestion_timestamp) = CURRENT_DATE()
""")

display(df_today)

# Show sample records
df_sample = spark.table(bronze_table).filter(
    col("ingestion_timestamp") >= current_date()
).limit(10)

display(df_sample)

logger.info("Load verification completed")